In [31]:
import pandas as pd

In [32]:
excel_file_path1 = 'C:/Users/ASUS/OneDrive/Documents/中信兄弟/Season_data/2023_non_trackman.xlsx'
excel_file_path2 = 'C:/Users/ASUS/OneDrive/Documents/中信兄弟/Season_data/2024_non_trackman.xlsx'
excel_file_path3 = 'C:/Users/ASUS/OneDrive/Documents/中信兄弟/程式碼/WOBA/RE.xlsx'
sheet_name = 'Sheet1'

In [33]:
non_track_2023 = pd.read_excel(excel_file_path1, sheet_name=sheet_name)
non_track_2024 = pd.read_excel(excel_file_path2, sheet_name=sheet_name)
RE = pd.read_excel(excel_file_path3, sheet_name=sheet_name)

In [34]:
col=['gameId','side','awayScores','homeScores','inning','paRound','paOrder','firstId','secondId','thirdId','result','outs','runsOnPlay','bName','pName','bTeamId']

In [35]:
non_track_2023= non_track_2023[col]
non_track_2024= non_track_2024[col]


In [36]:
non_track = pd.concat([non_track_2023, non_track_2024], ignore_index=True)

In [37]:
def calculate_base_state(row):
    state = 0
    if pd.notna(row['firstId']):
        state += 1
    if pd.notna(row['secondId']):
        state += 2
    if pd.notna(row['thirdId']):
        state += 4
    return state

# 应用函数创建新的 "壘包狀態" 列
non_track['壘包狀態'] = non_track.apply(calculate_base_state, axis=1)

In [38]:
def get_expected_runs(row):
    base_type = row["壘包狀態"]
    outs = row["outs"]
    #balls = row["balls"]
    #strikes = row["strikes"]
    
    matching_row = RE[(RE["壘包狀態"] == base_type) & 
                          (RE["outs"] == outs)] 
                          #(RE["Balls"] == balls) & 
                          #(RE["Strikes"] == strikes)]
    
    if not matching_row.empty:
        return matching_row.iloc[0]["RE"]
    else:
        return None

non_track["RE"] = non_track.apply(get_expected_runs, axis=1)


In [39]:
non_track = non_track.dropna(subset=['result'])

In [40]:
non_track['RE_change'] = non_track.groupby(['gameId','inning','side'])['RE'].diff().shift(-1)

last_bat_indexes = non_track.groupby(['gameId', 'inning', 'side'])['打者編號'].idxmax()

non_track.loc[last_bat_indexes, 'RE_change'] = -non_track['RE']

KeyError: 'Column not found: 打者編號'

In [ ]:
non_track['RE24']=non_track['RE_change']+non_track['runsOnPlay']

In [ ]:
non_track['打者編號'] = non_track.groupby(['gameId', 'side', 'inning']).cumcount() + 1
#non_track['打者編號'] = non_track.groupby(['gameId', 'side', 'inning']).cumcount() + 1

In [ ]:
non_track

In [ ]:
name_mapping = {
    '65630ee55559b6d7a6d5d200':'兄弟',
    '65630ee95559b6d7a6d5d201':'味全', 
    '65630eed5559b6d7a6d5d202':'樂天', 
    '65630ef25559b6d7a6d5d203':'統一',
    '65630ef65559b6d7a6d5d204':'富邦',
    '6569a9ddaf62a48d3f94b422':'台鋼',
    }

# 新增一列"球員"，根据"Pitcher"列的值匹配中文名
non_track ['球隊'] = non_track ['bTeamId'].map(name_mapping)
non_track

In [29]:
summary = non_track.groupby(['球隊','paOrder']).agg({"RE24": "mean"})

# 重新命名列名
#hitter_summary.rename(columns={"li值": "平均LI值", "WPA": "總WPA","打序": "PA"}, inplace=True)

# 輸出結果
summary

,RE24
paOrder,
1,-0.011754
2,-0.002732
3,0.005458
4,-0.008218
5,-0.004881
6,-0.028270
7,-0.036751
8,-0.045875
9,-0.041799


In [ ]:
summary.reset_index(inplace=True)

In [ ]:
summary

In [ ]:
#summary.to_excel('各棒次結果RE24.xlsx', index=True)

In [ ]:
summary2 = non_track.groupby(['paOrder']).agg({"paOrder": "count"})

# 輸出結果
summary2['PA/G']=summary2['paOrder']/300/2
summary2

In [ ]:
summary2.reset_index(drop=True, inplace=True)

In [ ]:
# 計算每一棒和下一棒打席數差異的百分比
summary2['Difference(%)'] = summary2['PA/G'].pct_change(1)

# 將Difference(%)移動一行，使其對應於當前paOrder
summary2['Difference(%)'] = summary2['Difference(%)'].shift(-1)

In [ ]:
summary2.rename(columns={"paOrder": "總數"}, inplace=True)
summary2['paOrder'] = range(1, 10)

In [ ]:
summary2

In [ ]:
def get_expected_runs(row):
    pa_type = row["paOrder"]

    matching_row = summary2[summary2["paOrder"] == pa_type]
    
    if not matching_row.empty:
        return matching_row.iloc[0]["Difference(%)"]
    else:
        return None

summary["Difference(%)"] = summary.apply(get_expected_runs, axis=1)

summary

In [ ]:
summary['AdjustedRE24'] = summary['RE24'] * (1 - (summary['Difference(%)']))
summary

In [ ]:
#summary.to_excel('各棒次結果RE24(打席數差異加成後).xlsx', index=True)